In [130]:
import nltk
import pandas as pd
import numpy as np
import emoji
import re
from nltk.corpus import stopwords
from textblob import Word
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn import svm 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [85]:
!pip install emoji

     |████████████████████████████████| 133kB 12.9MB/s 


In [108]:
tweets_df = pd.read_csv('Tweets.csv')

In [109]:
def pre_processing(df, stop_words):
    df['text'] = df['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    df['text'] = df['text'].str.replace('[^\w\s]', '')
    df['text'] = df['text'].str.replace('\d', '')
    df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words)) 
    df['text'] = df['text'].apply(lambda x: " ".join([Word(x).lemmatize() for x in x.split()]))
    return df

In [110]:
#change this
sentiment_ordering = ['negative', 'neutral', 'positive']
tweets_df['airline_sentiment'] = tweets_df['airline_sentiment'].apply(lambda x: sentiment_ordering.index(x))

In [116]:
#change this
tweets_df =tweets_df[['text','airline_sentiment']]
tweets_df.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,1
1,@VirginAmerica plus you've added commercials t...,2
2,@VirginAmerica I didn't today... Must mean I n...,1
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0


In [125]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [123]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [126]:
stop_words = stopwords.words('english')
tweets_df = pre_processing(tweets_df ,stop_words)

In [127]:
tweets_df.head()

,text,airline_sentiment
0,virginamerica dhepburn said,1
1,virginamerica plus youve added commercial expe...,2
2,virginamerica didnt today must mean need take ...,1
3,virginamerica really aggressive blast obnoxiou...,0
4,virginamerica really big bad thing,0


In [128]:
reviews = tweets_df.text[:]
sentiments = tweets_df.airline_sentiment[:]

In [132]:
vec = TfidfVectorizer(min_df = 4, max_df = 0.9)
vec_reviews = vec.fit_transform(reviews)


In [133]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {'alpha':[1,0.5,2]}
    }     
}

In [134]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(vec_reviews, sentiments)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.736749,"{'C': 1, 'kernel': 'linear'}"
1,logistic_regression,0.746380,{'C': 5}
2,naive_bayes_multinomial,0.728552,{'alpha': 0.5}
